In [ ]:
import os 
import bittensor as bt
import pandas as pd 
import numpy as np
import plotly.express as px
import wandb 

import torch 


In [123]:
def load_run(run_id: str, parent_location:str = "/opentensor-dev/folding-miners") -> pd.DataFrame:
    """
    Load the history of a specific run from WandB and return it as a DataFrame.

    This function connects to the WandB API, retrieves the history of the specified run,
    and returns it as a pandas DataFrame.

    Parameters:
    run_id (str): The unique identifier for the run.

    Returns:
    DataFrame: A pandas DataFrame containing the run's history.
    """
    api = wandb.Api()
    bt.logging.info(f"Loading run_id {run_id}")
    try:
        run = api.run(f"{parent_location}/runs/{run_id}")
        df = pd.DataFrame(list(run.scan_history()))
    except Exception as e:
        bt.logging.error(f"Error loading run data: {e}")
        raise

    return df

## looking at the metragraph

## wandb data

In [ ]:
df = load_run('87rnh7ua')

In [124]:
taostats = "imno16m7"
wildsage = '4tccezbn'
keith = "45cyt2vp"

runs = {
    'wildsage' : wildsage, 
    'taostats' : taostats, 
    'keith' : keith 
}


## Calculating ema scores

In [104]:
def calculate_weights(df):
    scores = torch.zeros(m.n, dtype=torch.float32) 
    for row in df.iterrows():
        response = row[1]
        if response.isna().sum() > 2:
            continue
        rewards = torch.FloatTensor(response.rewards)#, dtype=torch.float32)

        try:
            uids = torch.tensor(response.uids)
        except:
            print(uids)


        scattered_rewards: torch.FloatTensor = scores.scatter(0,uids,rewards)
        scores: torch.FloatTensor = 0.1 * scattered_rewards + (
            1 - 0.1
        ) * scores 

    return scores 
    

In [35]:
def calculate_scores(runs: dict, parent_location = "/opentensor-dev/folding-validators"):
    all_scores = []
    for name, run in runs.items(): 
        df = load_run(run, parent_location=parent_location)

        scores = calculate_weights(df)
        print(f"Scores for folding vali {name}: ", run)
        print(scores)
        print("")

        all_scores.append(scores)
    
    return all_scores 

In [36]:
all_scores = calculate_scores(runs = runs)

Scores for folding vali wildsage:  4tccezbn
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.5276, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.6513, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

In [37]:
# Start with an identity tensor of the same shape
result_tensor = torch.ones_like(all_scores[0])

# Multiply all tensors
for tensor in all_scores:
    result_tensor *= tensor

print(result_tensor)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


## Looking at network statistics

In [125]:
#calculating a single dataframe 
df = pd.DataFrame() 
for name, run in runs.items(): 
    samples = load_run(run, parent_location="opentensor-dev/folding-validators")
    samples['validator'] = name
    df = pd.concat([df, samples], axis = 0)
df 

,updated_at,best_hotkey,response_returned_files,_step,validator_search_status,active,hotkeys,response_status_codes,rewards,best_loss_at,...,hp_tries,max_time_no_improvement,ff,pdb,updated_count,pdb_id,_timestamp,block,forward_time,validator
0,2024-05-23T11:28:33,5CG5tKyYC1rvqMKi3JikYL7xTxmoAogLX6cSZeL8vGMbKrFr,"[[], [md_0_1.cpt, md_0_1.xtc, md_0_1_prev.cpt,...",0,True,True,[5HWTE4wvm3NEWoAVHefQfKLCYi9Y721mjkVLe9fWYxsdX...,"[408, 200, 200, 200, 503, 200, 503, 408, 408, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-23T10:56:05,...,0,3600.0,charmm27,1h2x,4.0,1h2x,1.716557e+09,3026417.0,NaN,wildsage
1,2024-05-24T18:26:02,5CFtZJPKv7nLvT1tQp7dpL1arrQgYEzshpmPhc4ZLbEVU38Y,"[[md_0_1_intermediate.gro, md_0_1.xtc, md_0_1....",1,True,True,[5CFtZJPKv7nLvT1tQp7dpL1arrQgYEzshpmPhc4ZLbEVU...,"[200, 503, 503, 408, 200, 200, 503, 200, 503, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-24T17:37:14,...,0,3600.0,charmm27,8djj,8.0,8djj,1.716557e+09,3026428.0,NaN,wildsage
2,2024-05-24T18:08:44,nan,"[[], [nvt.mdp, nvt.tpr, nvt.log, nvt.xtc, nvt....",2,True,True,[5EZ7raEJoz6xJPCGDjH1YZ5B5fNXyerZMPTK7hKFfikdb...,"[404, 200, 200, 200, 200, 408, 200, 408, 408, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaT,...,0,3600.0,charmm27,4for,0.0,4for,1.716557e+09,3026439.0,NaN,wildsage
3,2024-05-24T18:30:21,5DyCJ6P43VwGTYC3gqYB2S7wEBSno5jrV4QbnyszXRwJpEqm,"[[nvt.xtc, nvt.tpr, nvt.mdp, nvt.edr, nvt.log]...",3,True,True,[5DyCJ6P43VwGTYC3gqYB2S7wEBSno5jrV4QbnyszXRwJp...,"[200, 200, 408, 200, 200, 200, 200, 200, 503, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-24T18:30:21,...,0,3600.0,amber03,303d,1.0,303d,1.716557e+09,3026449.0,NaN,wildsage
4,2024-05-23T11:28:33,5CG5tKyYC1rvqMKi3JikYL7xTxmoAogLX6cSZeL8vGMbKrFr,"[[], [md_0_1.cpt, md_0_1.xtc, md_0_1_prev.cpt,...",4,True,True,[5HWTE4wvm3NEWoAVHefQfKLCYi9Y721mjkVLe9fWYxsdX...,"[408, 200, 200, 200, 503, 200, 503, 408, 408, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-23T10:56:05,...,0,3600.0,charmm27,1h2x,4.0,1h2x,1.716558e+09,3026477.0,NaN,wildsage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,2024-05-24T14:56:38,5DRY6eG3WLTP7d8QarhkhT9Z7sN2fDnJBwtetQYVtF9Tb2Ko,"[[nvt.mdp, nvt.tpr, nvt.log], [], [], [], [], ...",309,True,True,[5GH8ArTt6BM1xUeqxuKAaRTzL3EjZxfFKsJAKGzqrbHkD...,"[200, 503, 408, 503, 503, 408, 408, 408, 503, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-24T14:44:31,...,0,3600.0,amber03,2e1b,6.0,2e1b,1.716585e+09,3028761.0,NaN,keith
310,2024-05-24T21:17:21,5EFiiNVJMuSBYP7JaQVdcBGa2bdtLbcUSsYjrxtk2rWJh1Wo,"[[md_0_1.log, md_0_1.xtc, md_0_1_prev.cpt, md_...",310,True,True,[5EFiiNVJMuSBYP7JaQVdcBGa2bdtLbcUSsYjrxtk2rWJh...,"[200, 200, 200, 503, 200, 503, 200, 200, 408, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-24T20:56:21,...,0,3600.0,amber03,1qmv,15.0,1qmv,1.716585e+09,3028773.0,NaN,keith
311,2024-05-24T20:30:20,None,"[[], [], [nvt_prev.cpt, nvt.tpr, nvt.mdp, nvt....",311,True,True,[5DWr5BBBbhm5xVk6TXQyJtCFw7q4DeudMmm4faNN3PNGW...,"[503, 503, 200, 503, 200, 200, 200, 503, 200, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaT,...,0,3600.0,charmm27,2qjl,0.0,2qjl,1.716585e+09,3028776.0,NaN,keith
312,2024-05-24T12:23:27,5CFtZJPKv7nLvT1tQp7dpL1arrQgYEzshpmPhc4ZLbEVU38Y,"[[], [], [nvt.cpt, nvt.edr, nvt.xtc, nvt_inter...",312,True,True,[5EvRwaqzbb7VsbaPFaQrbnD8Ynq6L5Lv6L3n3YwW8e7F5...,"[503, 408, 200, 404, 200, 200, 200, 503, 503, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",2024-05-24T12:23:27,...,0,3600.0,amber03,4rbz,1.0,4rbz,1.716586e+09,3028801.0,NaN,keith


In [154]:
rt = df.response_times.explode() 
percent = f"{(rt[rt > 0] < 30).sum() / len(rt[rt > 0]):.3f}"
px.histogram(rt[rt > 0], 
             height = 600, width = 1000,
             log_y=False,
             title=f"Response Times, {100 * float(percent)}% of network < 30s", labels={'value':'Response Time (s)', 'count':'Count'}).show()

In [67]:
df['best_hotkey_short'] = df['best_hotkey'].apply(lambda x: str(x)[:8] if x is not np.nan else x)
df.groupby('validator').best_hotkey_short.value_counts(normalize=False)

validator  best_hotkey_short
keith      None                 79
           5CFtZJPK             32
           5GuMMDh8             29
           5DRY6eG3             24
           5Gma73Ah             13
           5EFiiNVJ             12
           5FqQmL4S             12
           5E9dNiY6             10
           5HjbTouu             10
           5DyCJ6P4              8
           5DZhLdJk              5
           5Dsug8Do              5
           5CfoMcMQ              3
           nan                   2
taostats   None                 12
           5F1d64Nc             11
           5G4dfqXw             11
           5EDDmDVq              9
           nan                   3
wildsage   None                 33
           5CG5tKyY             31
           5CfoMcMQ             10
           5G4dfqXw             10
           5HBh7LVE             10
           5E1c3as7              8
           5GuMMDh8              7
           5HNN4yU9              5
           nan            

## Calculating scores for Prompting

In [98]:
m = bt.metagraph(1)

bradleytf = "i551fano"
foundation = 'rgz2rx9f'
manifold = "jt2srgss"

runs = {
    'bradleytf' : bradleytf, 
    'foundation' : foundation, 
    'manifold' : manifold 
}


In [106]:
df = load_run(runs["foundation"], parent_location="opentensor-dev/alpha-validators") 
df 

,rouge_penalty_batch_time,rouge_reward_batch_time,challenge,reference,step,stream_results_exceptions,timings,system_prompt,_step,completions,...,ordinal_reward,ordinal_raw_reward,ordinal_reward_batch_time,ordinal_reward_extra_info.type,float_diff_reward_batch_time,float_diff_reward_timings,float_diff_raw_reward,float_diff_reward,float_diff_reward_extra_info.type,unexpected_errors
0,0.442612,0.408935,Could you assist me with translating the follo...,Valentina de Rachel é uma baía cheia de uma pe...,0.0,"[None, None, None, None, None, None, None, Non...","[1.305140495300293, 1.647301197052002, 2.39296...",This is a roleplaying game where you are imper...,0,[Valentina de Rachel é uma baía cheia de uma p...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.031771,"\n\nWhat year did Bernardini, the sire of Rach...","\n\nAccording to the context, Bernardini, the ...",1.0,"[None, None, None, None, None, None, None, Non...","[3.5550827980041504, 2.8413479328155518, 2.896...",This is a roleplaying game where you are imper...,1,"[According to the context, Bernardini, the sir...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.136734,0.175525,"\n\nlook, i don't got all day. Can you just te...","\n\nluke jensen, a former french open doubles ...",2.0,"[None, None, None, None, None, None, None, Non...","[8.604170560836792, 8.907668113708496, 9.01988...",This is a roleplaying game where you are imper...,2,[Ethan Hawke is a versatile actor known for hi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.084186,0.656740,\n\nwhat exactly does a block development offi...,\n\nthe primary role of the block development ...,3.0,"[None, None, None, None, None, None, None, Non...","[6.525025129318237, 8.855193614959717, 7.75953...",This is a roleplaying game where you are imper...,3,[A Block Development Officer (BDO) in the admi...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.136458,0.816375,\n\ncan you elaborate on the specific mechanis...,\n\nthe block development officer (bdo) in sin...,3.0,"[None, None, None, None, None, None, None, Non...","[3.190364122390747, 4.420850992202759, 4.93069...",This is a roleplaying game where you are imper...,4,[The block development officer (bdo) in Sinapa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5740,0.226106,0.991524,\n\nwhat would be the most likely outcome if r...,"\n\nif richard nixon's infamous ""enemies list""...",3281.0,"[None, None, None, None, None, None, None, Non...","[3.039862632751465, 4.209098815917969, 4.41196...",This is a roleplaying game where you are imper...,5740,"[Recreating Richard Nixon's \""enemies list\"" t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5741,0.595442,0.553970,How about translating this text into English f...,Epithelia turn at some of the fastest rates in...,3282.0,"[None, None, None, None, None, None, None, Non...","[1.4291856288909912, 1.2141830921173096, 7.260...",This is a roleplaying game where you are imper...,5741,[Epithelia turn into some of the fastest rates...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5742,0.138121,0.962059,\n\ncan you elaborate on the specific mechanis...,\n\ni'd be happy to elaborate on the specific ...,3282.0,"[None, None, None, None, None, None, None, Non...","[2.7513341903686523, 3.2897555828094482, 3.944...",This is a roleplaying game where you are imper...,5742,[When many cells accumulate in the epithelial ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5743,NaN,NaN,Hola! I'd like to know the sentiment of this ...,neutral,3283.0,"[None, None, None, None, None, None, None, Non...","[1.9421577453613281, 1.44270920753479, 3.50372...",This is a roleplaying game where you are imper...,5743,"[Neutral, neutral, neutral, Neutral, Neutral, ...",...,"[0.9999990463256836, 0.9999990463256836, 0.999...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",0.001189,ordinal,NaN,NaN,NaN,NaN,NaN,NaN


## Scrap

In [121]:
val = 1
vals = []
for ii in range(10):
    vals.append(val)
    val = val * 0.1 
    
px.line(vals, log_y = False)

In [110]:
df.rewards.explode().value_counts()

rewards
1.000000    41689
0.000000    11388
0.500000     1839
0.957143      279
0.950000      215
            ...  
0.376795        1
0.393208        1
0.395071        1
0.353200        1
0.320385        1
Name: count, Length: 135658, dtype: int64